In [1]:
import numpy as np

In [2]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as pp

In [9]:
# rawdata is the excel contains data of NAISC and 'Technology - In Lab v In the Market',
#Broad? (0 = Detailed)', 'Unconventional', 'Relevance',
#Manual/Cognitive', 'Routine/Non-Routine', 'Hardware Disruptor',
#Software Disruptor', 'OxfordStudyPercentage',
#Singular Task v Multi Task'
rawdata=pd.read_csv("E:/research/Automation in RE/datasummary/rawdatanaics.csv")

# all the occ_code from Onet, but omit the last two digits, change 8 to 6 digits so that it alligns with NAISC
occ_onet=pd.read_csv("E:/research/Automation in RE/datasummary/Occupation Data.csv")

# Onet code does not end with 00,so by get the overlapping, we exclude those general groups
del_occ_endwith_00_and_in_Onet=rawdata.loc[rawdata['occ_code'].isin(occ_onet['O*NET-SOC Code'])]

D:\anaconda\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15,48) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
occ_onet.columns
del_occ_endwith_00_and_in_Onet.columns

Index(['year', 'naics', 'naics_title', 'title_code', 'dominant_lifecycle',
       'occ_code', 'occ_code_mid', 'occ_title', 'occ_title_code',
       'occ_title_mid', 'year.1', 'naics.1', 'naics_title.1', 'occ_code.1',
       'occ_title.1', 'group', 'tot_emp_num', 'emp_prse', 'pct_total',
       'pct_rpt', 'h_mean', 'a_mean', 'mean_prse', 'h_pct10', 'h_pct25',
       'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25', 'a_median_num',
       'a_pct75', 'a_pct90', 'naics_codedescription', 'year-naics-occ_code',
       'year-naics-occ_code.1', 'match', 'year.2', 'naics.2', 'naics_title.2',
       'occ_code.2', 'title_code.1', 'dominant_lifecycle.1', 'occ_title.2',
       'occ_title_code.1', 'occ_title_mid.1', 'Task',
       'Lifecycle of a Building', 'group.1', 'naics_codedescription.1',
       'keepsy', 'Technology - In Lab v In the Market',
       'Broad? (0 = Detailed)', 'Unconventional', 'Relevance',
       'Manual/Cognitive', 'Routine/Non-Routine', 'Hardware Disruptor',
       'Sof

In [13]:
occ_onet.columns = ['O*NET-SOC Code', 'XX', 'Title', 'Description']
occ_onet=occ_onet.drop(['XX'],axis=1)
del_occ_endwith_00_and_in_Onet.shape


ValueError: Length mismatch: Expected axis has 3 elements, new values have 4 elements

In [14]:
occ_onet.columns


Index(['O*NET-SOC Code', 'Title', 'Description'], dtype='object')

In [16]:
occ_onet.head(5)
occ_onet.shape

(1110, 3)

In [17]:
# Generate the features and years we need for labor-economic matric
rowname=pd.read_csv("E:/research/Automation in RE/datasummary/laborecon-rowname.csv")
for string in rowname.naics_title:
    print ("'"+str(string)+"',")
for num in range(2007,2017):
    print("'"+str(num)+"',")

'Residential Building Construction',
'Nonresidential Building Construction',
'Land Subdivision',
'Foundation, Structure, and Building Exterior Contractors',
'Building Equipment Contractors',
'Building Finishing Contractors',
'Other Specialty Trade Contractors',
'Lessors of Real Estate',
'Offices of Real Estate Agents and Brokers',
'Activities Related to Real Estate',
'Architectural, Engineering, and Related Services',
'Specialized Design Services',
'Services to Buildings and Dwellings',
'nan',
'2007',
'2008',
'2009',
'2010',
'2011',
'2012',
'2013',
'2014',
'2015',
'2016',


In [7]:
len(set(rawdata['occ_code']))

1058

In [20]:
# Get the 558 occupations that we target in
target_occ=set(del_occ_endwith_00_and_in_Onet['occ_code'])
target_occ=list(target_occ)
target=pd.DataFrame({'target_occ': target_occ})

In [23]:
del_occ_endwith_00_and_in_Onet.columns
#drop the colomuns that contain duplicate information
del_duplicated_column=del_occ_endwith_00_and_in_Onet.drop(['title_code','occ_title_code', 'occ_title_mid', 'year.1', 'naics.1', 'naics_title.1', 'occ_code.1', 'occ_title.1', 'group', 'naics_codedescription', 'year-naics-occ_code', 'year-naics-occ_code.1','year.2', 'naics.2', 'naics_title.2', 'occ_code.2', 'title_code.1', 'dominant_lifecycle.1', 'occ_title.2', 'occ_title_code.1', 'occ_title_mid.1','group.1', 'naics_codedescription.1', 'keepsy'],axis=1)
# the columns # decrease to 37 from 61
del_duplicated_column.shape

      
    


(22026, 37)

(22026,)

In [11]:
# try to extract data by occ_code 
dx = del_duplicated_column[del_duplicated_column['occ_code'] == '11-1011']
dx[['year','naics_title', 'tot_emp_num']].head(3)

,year,naics_title,tot_emp_num
2,2007,Residential Building Construction,2700.0
203,2007,Nonresidential Building Construction,3580.0
405,2007,Land Subdivision,550.0


In [25]:
# assign the matrix of "year" and "naics_code" to a specific occ_code, call it 'labour_economy_data'
# first, create a function that generate the lb_econ_matrix for an occ_code
def lb_econ (occ_code,field,data):
    # this function yield a data frame for a given occ_code and a field
    # get all unique naics titles in the data
    feature_list = data['naics_title'].unique()
    # get all unique year in the data
    years = sorted(data['year'].unique())    
    mt_result = pd.DataFrame(0.00, index = years, columns = feature_list)   
    # filter out all rows and columns that have the given occ_code and fields
    df_occ = data[data['occ_code'] == occ_code][['year','naics_title', field]]
    for index, row in df_occ.iterrows():
        try:
            mt_result.loc[row['year']][row['naics_title']] = float(row[field])
        except:
            mt_result.loc[row['year']][row['naics_title']] = np.nan
    return mt_result.round(2)



In [26]:
data1=del_duplicated_column
aa=lb_econ ('11-1011','tot_emp_num',data1)
aa.head()

,Residential Building Construction,Nonresidential Building Construction,Land Subdivision,"Foundation, Structure, and Building Exterior Contractors",Building Equipment Contractors,Building Finishing Contractors,Other Specialty Trade Contractors,Lessors of Real Estate,Offices of Real Estate Agents and Brokers,Activities Related to Real Estate,"Architectural, Engineering, and Related Services",Specialized Design Services,Services to Buildings and Dwellings
2007,2700.0,3580.0,550.0,1780.0,5170.0,1420.0,1580.0,930.0,910.0,1470.0,6280.0,710.0,2300.0
2008,2380.0,3900.0,460.0,1780.0,4850.0,1400.0,1530.0,970.0,810.0,1590.0,5800.0,610.0,2220.0
2009,1780.0,3730.0,490.0,1690.0,4750.0,1190.0,1400.0,920.0,690.0,1600.0,5670.0,590.0,2140.0
2010,1320.0,3030.0,410.0,1430.0,3980.0,1010.0,1040.0,1010.0,550.0,1550.0,5050.0,400.0,1700.0
2011,1080.0,2780.0,400.0,1400.0,4150.0,890.0,910.0,990.0,530.0,1410.0,4870.0,420.0,1590.0


In [28]:
# the lb_Econ_features contains all the fields' name that one can choose from, there're 17 all together
lb_econ_features=['occ_code','tot_emp_num', 'emp_prse', 'pct_total', 'pct_rpt', 'h_mean', 'a_mean', 'mean_prse', 'h_pct10', 'h_pct25', 'h_median', 'h_pct75', 'h_pct90', 'a_pct10', 'a_pct25', 'a_median_num', 'a_pct75', 'a_pct90']
#try to input another field name called 'emp_prse'





,Residential Building Construction,Nonresidential Building Construction,Land Subdivision,"Foundation, Structure, and Building Exterior Contractors",Building Equipment Contractors,Building Finishing Contractors,Other Specialty Trade Contractors,Lessors of Real Estate,Offices of Real Estate Agents and Brokers,Activities Related to Real Estate,"Architectural, Engineering, and Related Services",Specialized Design Services,Services to Buildings and Dwellings
2007,6.9,3.8,18.1,6.3,2.9,5.3,6.0,6.5,8.7,6.1,3.0,9.5,6.2
2008,10.5,6.2,12.8,6.2,2.8,7.0,5.2,6.4,7.9,7.5,2.9,7.7,7.7
2009,8.4,5.0,8.3,5.6,3.0,8.9,5.3,9.0,6.4,7.3,2.9,6.1,6.8
2010,9.9,5.7,7.5,5.1,3.4,6.9,5.5,9.0,9.0,7.2,2.9,7.5,5.8
2011,8.7,6.9,7.4,5.0,3.7,5.6,6.5,9.2,10.4,7.7,3.3,10.8,5.7


In [32]:
# Now we start to add O*net features to the 557 occupations
# Pause here because there're problems to match the 6-digit target occupations to 8-digit Onet occ_code
# In conclusion
# We not have 558 6-digit occupations that related to Real Estate
# The NAISC data are store in 'data1'
# The function to get labor economic data for each occupation is called 'lb_Econ'
# Try lb_econ( ('11-1011','tot_emp_num',data1))
bb=lb_econ('11-1011','emp_prse',data1)
bb.head()

,Residential Building Construction,Nonresidential Building Construction,Land Subdivision,"Foundation, Structure, and Building Exterior Contractors",Building Equipment Contractors,Building Finishing Contractors,Other Specialty Trade Contractors,Lessors of Real Estate,Offices of Real Estate Agents and Brokers,Activities Related to Real Estate,"Architectural, Engineering, and Related Services",Specialized Design Services,Services to Buildings and Dwellings
2007,6.9,3.8,18.1,6.3,2.9,5.3,6.0,6.5,8.7,6.1,3.0,9.5,6.2
2008,10.5,6.2,12.8,6.2,2.8,7.0,5.2,6.4,7.9,7.5,2.9,7.7,7.7
2009,8.4,5.0,8.3,5.6,3.0,8.9,5.3,9.0,6.4,7.3,2.9,6.1,6.8
2010,9.9,5.7,7.5,5.1,3.4,6.9,5.5,9.0,9.0,7.2,2.9,7.5,5.8
2011,8.7,6.9,7.4,5.0,3.7,5.6,6.5,9.2,10.4,7.7,3.3,10.8,5.7


In [16]:
for index, row in target.iterrows():
     print('INSERT INTO target (occ_code) VALUES ("'+str(row[0])+'");')

INSERT INTO target (occ_code) VALUES ("47-4099");
INSERT INTO target (occ_code) VALUES ("17-2171");
INSERT INTO target (occ_code) VALUES ("51-2021");
INSERT INTO target (occ_code) VALUES ("51-9123");
INSERT INTO target (occ_code) VALUES ("19-3032");
INSERT INTO target (occ_code) VALUES ("39-1021");
INSERT INTO target (occ_code) VALUES ("49-9011");
INSERT INTO target (occ_code) VALUES ("41-9021");
INSERT INTO target (occ_code) VALUES ("49-3023");
INSERT INTO target (occ_code) VALUES ("11-9161");
INSERT INTO target (occ_code) VALUES ("47-4011");
INSERT INTO target (occ_code) VALUES ("51-2011");
INSERT INTO target (occ_code) VALUES ("49-9095");
INSERT INTO target (occ_code) VALUES ("51-9011");
INSERT INTO target (occ_code) VALUES ("47-2111");
INSERT INTO target (occ_code) VALUES ("51-6011");
INSERT INTO target (occ_code) VALUES ("47-5049");
INSERT INTO target (occ_code) VALUES ("47-5071");
INSERT INTO target (occ_code) VALUES ("47-2061");
INSERT INTO target (occ_code) VALUES ("31-9096");


In [20]:
# Add the new table called 'target' to the SQL database 'combinded', I did it in SQLite.
# Add target occ_code value to the table called'target',using the queries printed above, I did it in SQLite.
import sqlite3
conn = sqlite3.connect('E:/research/Automation in RE/data-onet/db_23_0_mysql/combined.sqlite')
cur = conn.cursor()
query = "SELECT * FROM occupation_Data";
occdata = pd.read_sql_query(query,conn)
len(occdata)


1109

,onetsoc_code,title,description


0       11-1011
1       11-1011
2       11-1021
3       11-1031
4       11-2011
5       11-2011
6       11-2021
7       11-2022
8       11-2031
9       11-3011
10      11-3021
11      11-3031
12      11-3031
13      11-3031
14      11-3051
15      11-3051
16      11-3051
17      11-3051
18      11-3051
19      11-3051
20      11-3051
21      11-3061
22      11-3071
23      11-3071
24      11-3071
25      11-3071
26      11-3111
27      11-3121
28      11-3131
29      11-9013
         ...   
1079    53-7061
1080    53-7062
1081    53-7063
1082    53-7064
1083    53-7071
1084    53-7072
1085    53-7073
1086    53-7081
1087    53-7111
1088    53-7121
1089    53-7199
1090    55-1011
1091    55-1012
1092    55-1013
1093    55-1014
1094    55-1015
1095    55-1016
1096    55-1017
1097    55-1019
1098    55-2011
1099    55-2012
1100    55-2013
1101    55-3011
1102    55-3012
1103    55-3013
1104    55-3014
1105    55-3015
1106    55-3016
1107    55-3017
1108    55-3018
Name: code, Length: 1109

In [ ]:
q1="SELECT onetsoc_code,title,description FROM occupation_Data,target WHERE occupation_Data.onetsoc_code==target.occ_code";
title_des=pd.read_sql_query(q1,conn)
title_des

In [125]:
# add the features to the target occ, i.e. to the 'target_df'
    
target_df=pd.DataFrame(index = target_occ, columns = occdata.columns[1:])
target_df.head(5)


,title,description
47-2231,NaN,NaN
51-4011,NaN,NaN
53-7032,NaN,NaN
33-9091,NaN,NaN
31-9092,NaN,NaN


819    Assemble, install, or maintain solar photovolt...
Name: description, dtype: object

,title,description
47-2231,819 Solar Photovoltaic Installers Name: tit...,"819 Assemble, install, or maintain solar ph..."
51-4011,928 Computer-Controlled Machine Tool Operat...,928 Operate computer-controlled machines or...
53-7032,1075 Excavating and Loading Machine and Dra...,1075 Operate or tend machinery equipped wit...
33-9091,"595 Crossing Guards Name: title, dtype: object",595 Guide or control vehicular or pedestria...
31-9092,"555 Medical Assistants Name: title, dtype: ...",555 Perform administrative and certain clin...


,code,title,description
0,11-1011,Chief Executives,Determine and formulate policies and provide o...
1,11-1011,Chief Sustainability Officers,"Communicate and coordinate with management, sh..."
2,11-1021,General and Operations Managers,"Plan, direct, or coordinate the operations of ..."
3,11-1031,Legislators,"Develop, introduce or enact laws and statutes ..."
4,11-2011,Advertising and Promotions Managers,"Plan, direct, or coordinate advertising polici..."


(558, 2)

In [ ]:
# Transform the 8-digit Onet-occupation code into 6-digit thus coincides with the NAISC code.
query = "SELECT * FROM occupation_Data";
occdata = pd.read_sql_query(query,conn)
occdata.columns=['code','title','description']
for i in range(len(occdata['code'])):
    occdata['code'][i]=occdata['code'][i][:7]
occdata['code']
for index, row in occdata.iterrows():
     print('UPDATE occupation_data SET onetsoc_code ='+str(row[0])+'WHERE ");')